In [1]:
using JSON
using DataFrames
using JuMP, Gurobi
using DelimitedFiles

In [2]:
dict = Dict()
open("day_data.json", "r") do f
    global dict
    dict=JSON.parse(f)  
end

travel_time = readdlm("travel_times.csv", ';')[2:54,2:54]
n = length(dict["bookings"]) 
k = length(dict["shifts"]) 

price = zeros(n)
for i = 1:n
    price[i] = dict["bookings"][i]["price"]
end

In [3]:
cost = zeros(2*n+2,2*n+2,k)

for l = 1:k
    for i = 1:2*n+2
        for j = 1:2*n+2
            
            if 2 <= i <= n+1
                depart = parse(Int,dict["bookings"][i-1]["jobs"][1]["station"][2:end])
            elseif n+2 <= i <= 2*n+1
                depart = parse(Int,dict["bookings"][i-1-n]["jobs"][2]["station"][2:end])
            elseif i == 1
                depart = 0
            elseif i == 2*n+2
                depart = 0
            end
            
            if 2 <= j <= n+1
                arrivee = parse(Int,dict["bookings"][j-1]["jobs"][1]["station"][2:end])
            elseif n+2 <= j <= 2*n+1
                arrivee = parse(Int,dict["bookings"][j-1-n]["jobs"][2]["station"][2:end])
            elseif j == 1
                arrivee = 0
            elseif j == 2*n+2
                arrivee = 0
            end
                 
            cost[i,j,l] = travel_time[depart+1,arrivee+1]
            
        end
    end
end        

In [4]:
m = Model(with_optimizer(Gurobi.Optimizer))
set_optimizer_attributes(m)

# pour chaque chauffeur k la matrice des trajets (i,j,k)
@variable(m, x[1:2*n+2,1:2*n+2,1:k], Bin) 

@variable(m, u[1:2*n], Int)

@variable(m, v[1:k,1:2], Int)  #temps où la station de pickup i est servie par le chauffeur k


@variable(m, w[1:2*n+2], Int)  #quantité que le chauffeur k a lorsqu'il visite le point i

@variable(m, r[1:n], Int)  #temps que prends le client au point i pour finir sa course 

┌ Warning: `with_optimizer` is deprecated. Adapt the following example to update your code:
│ `with_optimizer(Ipopt.Optimizer)` becomes `Ipopt.Optimizer`.
│   caller = top-level scope at In[4]:1
└ @ Core In[4]:1


Academic license - for non-commercial use only


25-element Array{VariableRef,1}:
 r[1] 
 r[2] 
 r[3] 
 r[4] 
 r[5] 
 r[6] 
 r[7] 
 r[8] 
 r[9] 
 r[10]
 r[11]
 r[12]
 r[13]
 r[14]
 r[15]
 r[16]
 r[17]
 r[18]
 r[19]
 r[20]
 r[21]
 r[22]
 r[23]
 r[24]
 r[25]

In [5]:
for i = 2:n+1
        @constraint(m, sum(x[i,:,:])==1)
end

In [7]:
for l = 1:k   
    @constraint(m, sum(x[1,:,l])==1) #départ du dépot
    @constraint(m, sum(x[:,2*n+2,l])==1) # retour au depot
    @constraint(m, v[l,2] <= dict["shifts"][l]["jobs"][2]["timeDate"]) #respect des shifts
    @constraint(m, v[l,1] >= dict["shifts"][l]["jobs"][1]["timeDate"])  #respect des shifts
end

In [8]:
for i = 2:n+1
    for l = 1:k
        @constraint(m, sum(x[i,:,l]) - sum(x[i+n,:,l]) == 0)  # 4
        @constraint(m, sum(x[:,i,l]) - sum(x[i,:,l]) == 0) #5
        @constraint(m, sum(x[:,i+n,l]) - sum(x[i+n,:,l]) == 0) #5
    end
end

In [9]:
M = zeros(2*n+2,2*n+2)


for i = 1:2*n+2
    for j = 1:2*n+2

        if 2 <= i <= n+1
            d = dict["bookings"][i-1]["jobs"][1]["duration"]
        elseif n+2 <= i <= 2*n+1
            d = dict["bookings"][i-n-1]["jobs"][2]["duration"]
        elseif i == 1
            d = 0
        elseif i == 2*n+2
            d = 0
        end

        if 2 <= i <= n+1
            depart = parse(Int,dict["bookings"][i-1]["jobs"][1]["station"][2:end])
        elseif n+2 <= i <= 2*n+1
            depart = parse(Int,dict["bookings"][i-1-n]["jobs"][2]["station"][2:end])
        elseif i == 1
            depart = 0
        elseif i == 2*n+2
            depart = 0
        end

        if 2 <= j <= n+1
            arrivee = parse(Int,dict["bookings"][j-1]["jobs"][1]["station"][2:end]) 
        elseif n+2 <= j <= 2*n+1
            arrivee = parse(Int,dict["bookings"][j-1-n]["jobs"][2]["station"][2:end])
        elseif j == 1
            arrivee = 0
        elseif j == 2*n+2
            arrivee = 0
        end

        if 2 <= j <= n+1
            begin_date = dict["bookings"][j-1]["jobs"][1]["timeWindowBeginDate"]
        elseif n+2 <= j <= 2*n+1
            begin_date = dict["bookings"][j-1-n]["jobs"][2]["timeWindowBeginDate"]
        elseif j == 1
            begin_date = 31200
        elseif j == 2*n+2
            begin_date = 31200
        end

        if 2 <= i <= n+1
            end_date = dict["bookings"][i-1]["jobs"][1]["timeWindowEndDate"]
        elseif n+2 <= i <= 2*n+1
            end_date = dict["bookings"][i-1-n]["jobs"][2]["timeWindowEndDate"]
        elseif i == 1
            end_date = 64548
        elseif i == 2*n+2
            end_date = 64548
        end

        M[i,j] = max(0,(end_date + travel_time[depart+1,arrivee+1] + d - begin_date))

        for l = 1:k
            if 2 <= i <= 2*n+1
                if 2 <= j <= 2*n+1
                    @constraint(m, u[j-1] >= u[i-1] + d + travel_time[depart+1,arrivee+1] - M[i,j]*(1 - x[i,j,l]))
                elseif j == 1
                    @constraint(m,  v[l,1] >= u[i-1] + d + travel_time[depart+1,arrivee+1] - M[i,j]*(1 - x[i,j,l]))
                elseif j == 2*n+2
                    @constraint(m,  v[l,2] >= u[i-1] + d + travel_time[depart+1,arrivee+1] - M[i,j]*(1 - x[i,j,l]))
                end
            elseif i == 1
                if 2 <= j <= 2*n+1
                    @constraint(m, u[j-1] >= v[l,1] + d + travel_time[depart+1,arrivee+1] - M[i,j]*(1 - x[i,j,l]))
                elseif j == 1
                    @constraint(m,  v[l,1] >= v[l,1] + d + travel_time[depart+1,arrivee+1] - M[i,j]*(1 - x[i,j,l]))
                elseif j == 2*n+2
                    @constraint(m,  v[l,2] >= v[l,1] + d + travel_time[depart+1,arrivee+1] - M[i,j]*(1 - x[i,j,l]))
                end
            elseif i == 2*n+2
                if 2 <= j <= 2*n+1
                    @constraint(m, u[j-1] >= v[l,2] + d + travel_time[depart+1,arrivee+1] - M[i,j]*(1 - x[i,j,l]))
                elseif j == 1
                    @constraint(m,  v[l,1] >= v[l,2] + d + travel_time[depart+1,arrivee+1] - M[i,j]*(1 - x[i,j,l]))
                elseif j == 2*n+2
                    @constraint(m,  v[l,2] >= v[l,2] + d + travel_time[depart+1,arrivee+1] - M[i,j]*(1 - x[i,j,l]))
                end
            end
        end

    end
end


In [10]:
W = zeros(2*n+2,2*n+2)

capacity = dict["shifts"][1]["capacity"]

for i = 1:2*n+2
    for j = 1:2*n+2
        if 2<= i <= n+1
            q_i = dict["bookings"][i-1]["passengers"]
        elseif n+2 <= i <= 2*n+1
            q_i = -dict["bookings"][i-1-n]["passengers"]
        elseif i == 1
            q_i = 0
        elseif i == 2*n+2
            q_i = 0
        end

        if 2<= j <= n+1
            q_j = dict["bookings"][j-1]["passengers"]
        elseif n+2 <= j <= 2*n+1
            q_j = -dict["bookings"][j-1-n]["passengers"]
        elseif j == 1
            q_j = 0
        elseif j == 2*n+2
            q_j = 0
        end

        W[i,j] = min(capacity, capacity + q_i)
        for l = 1:k
            @constraint(m, w[j] >= w[i]  + q_j - W[i,j]*(1 - x[i,j,l]))
        end
    end
end


In [11]:

for i = 2:n+1
    d = dict["bookings"][i-1]["jobs"][1]["duration"]
    @constraint(m, r[i-1] == (u[n+i-1] - u[i-1] - d)) 
end

In [12]:

for i = 1:2*n+2
    if 2<= i <= n+1
        q = dict["bookings"][i-1]["passengers"]
    elseif n+2 <= i <= 2*n+1
        q = -dict["bookings"][i-1-n]["passengers"]
    elseif i == 1
        q = 0
    elseif i == 2*n+2
        q = 0
    end

    if q >= 0
        @constraint(m, q <= w[i] <= capacity) 
    else
        @constraint(m, 0 <= w[i] <= capacity + q ) 
    end

end    

In [13]:
for i = 1:2*n+2
    if 2 <= i <= n+1
        begin_date = dict["bookings"][i-1]["jobs"][1]["timeWindowBeginDate"]
        end_date = dict["bookings"][i-1]["jobs"][1]["timeWindowEndDate"]
        @constraint(m, begin_date <= u[i-1] <= end_date) 
    elseif n+2 <= i <= 2*n+1
        begin_date = dict["bookings"][i-1-n]["jobs"][2]["timeWindowBeginDate"]
        end_date = dict["bookings"][i-1-n]["jobs"][2]["timeWindowEndDate"]
        @constraint(m, begin_date <= u[i-1] <= end_date) 
    end
end        

In [14]:
for i = 2:n+1
    depart = parse(Int,dict["bookings"][i-1]["jobs"][1]["station"][2:end])
    arrivee = parse(Int,dict["bookings"][i-1]["jobs"][2]["station"][2:end])
    L = dict["bookings"][i-1]["maximumDuration"]
    @constraint(m, travel_time[depart+1,arrivee+1]<= r[i-1] <= L) 
end       

In [15]:
for i = 1:2*n+2
    @constraint(m, sum(x[i,i,:])==0)
end

In [16]:
@constraint(m, sum(x[2*n+2,:,:])==0)


x[52,1,1] + x[52,2,1] + x[52,3,1] + x[52,4,1] + x[52,5,1] + x[52,6,1] + x[52,7,1] + x[52,8,1] + x[52,9,1] + x[52,10,1] + x[52,11,1] + x[52,12,1] + x[52,13,1] + x[52,14,1] + x[52,15,1] + x[52,16,1] + x[52,17,1] + x[52,18,1] + x[52,19,1] + x[52,20,1] + x[52,21,1] + x[52,22,1] + x[52,23,1] + x[52,24,1] + x[52,25,1] + x[52,26,1] + x[52,27,1] + x[52,28,1] + x[52,29,1] + x[52,30,1] + x[52,31,1] + x[52,32,1] + x[52,33,1] + x[52,34,1] + x[52,35,1] + x[52,36,1] + x[52,37,1] + x[52,38,1] + x[52,39,1] + x[52,40,1] + x[52,41,1] + x[52,42,1] + x[52,43,1] + x[52,44,1] + x[52,45,1] + x[52,46,1] + x[52,47,1] + x[52,48,1] + x[52,49,1] + x[52,50,1] + x[52,51,1] + x[52,52,1] + x[52,1,2] + x[52,2,2] + x[52,3,2] + x[52,4,2] + x[52,5,2] + x[52,6,2] + x[52,7,2] + x[52,8,2] + x[52,9,2] + x[52,10,2] + x[52,11,2] + x[52,12,2] + x[52,13,2] + x[52,14,2] + x[52,15,2] + x[52,16,2] + x[52,17,2] + x[52,18,2] + x[52,19,2] + x[52,20,2] + x[52,21,2] + x[52,22,2] + x[52,23,2] + x[52,24,2] + x[52,25,2] + x[52,26,2] + x[52

In [17]:
for l = 1:k
    @constraint(m, sum(x[2:n+1,:,l].* price) <= 6000 )
end

In [18]:
@objective(m, Min, sum(x.*cost) )

optimize!(m)
println("Objective value: ", objective_value(m))

Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 44166 rows, 21902 columns and 205281 nonzeros
Model fingerprint: 0x99e5a384
Variable types: 127 continuous, 21775 integer (21632 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+04]
  Objective range  [8e+01, 1e+03]
  Bounds range     [1e+00, 6e+04]
  RHS range        [1e+00, 6e+04]
Presolve removed 37939 rows and 17139 columns
Presolve time: 1.26s
Presolved: 6227 rows, 4763 columns, 34711 nonzeros
Variable types: 0 continuous, 4763 integer (4635 binary)

Root relaxation: objective 1.723888e+04, 1708 iterations, 0.14 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 17284.9109    0  203          - 17284.9109      -     -    1s
     0     0 18539.7863    0  196          - 18539.7863      -     -    2s
     0     0 18671.3750    0  

In [53]:
output = Dict()
shifts = []
for l = 1:k
    temp_dict = Dict()
    push!(temp_dict, "id" => dict["shifts"][l]["id"])
    jobs = []
    ## i = 1
    temp_dict2 = Dict()

    push!(temp_dict2, "id" => dict["shifts"][l]["jobs"][1]["id"])
    push!(temp_dict2, "time" => e[l,1])

    
    if isempty(temp_dict2) == false
        JSON.print(temp_dict2)
        push!(jobs, temp_dict2)
    end
    
    
    temp_dict2 = Dict()
    push!(temp_dict2, "id" => dict["shifts"][l]["jobs"][2]["id"])
    push!(temp_dict2, "time" => e[l,2])

    
    if isempty(temp_dict2) == false
        JSON.print(temp_dict2,4)
        push!(jobs, temp_dict2)
    end
    
    
    for i = 2:n+1
        temp_dict2 = Dict()
        for j = 2:2*n+1
            if a[i,j,l] == 1
                push!(temp_dict2, "id" => dict["bookings"][i-1]["jobs"][1]["id"])
                push!(temp_dict2, "time" => c[i-1])
            end
        end
        
        if isempty(temp_dict2) == false
            JSON.print(temp_dict2,4)
            push!(jobs, temp_dict2)
        end
    end
    
    for i = n+2:2*n+1
        temp_dict2 = Dict()
        for j = 2:2*n+2
            if a[i,j,l] == 1
                push!(temp_dict2, "id" => dict["bookings"][i-1-n]["jobs"][2]["id"])
                push!(temp_dict2, "time" => c[i-1])
            end
        end
        if isempty(temp_dict2) == false
            JSON.print(temp_dict2,4)
            push!(jobs, temp_dict2)
        end
    end
    push!(temp_dict, "jobs" => jobs)
    push!(shifts, temp_dict)
end
push!(output, "shifts"=>shifts)
push!(output, "route_cost" => objective_value(m))
push!(output, "nb_assigned_bookings" => 25)
json_string = JSON.json(output)

{"time":45000.0,"id":-9714126}{
    "time": 58800.0,
    "id": -9714125
}
{
    "time": 46800.0,
    "id": 25139347
}
{
    "time": 49942.0,
    "id": 25060099
}
{
    "time": 49702.0,
    "id": 24244107
}
{
    "time": 46325.0,
    "id": 24601133
}
{
    "time": 47394.0,
    "id": 25139348
}
{
    "time": 51273.0,
    "id": 25060100
}
{
    "time": 50578.0,
    "id": 24244108
}
{
    "time": 48905.0,
    "id": 24601134
}
{"time":31836.0,"id":-9714087}{
    "time": 41400.0,
    "id": -9714086
}
{
    "time": 37820.0,
    "id": 24102822
}
{
    "time": 36300.0,
    "id": 24758232
}
{
    "time": 32760.0,
    "id": 24873049
}
{
    "time": 36230.0,
    "id": 24861548
}
{
    "time": 38374.0,
    "id": 24102823
}
{
    "time": 36934.0,
    "id": 24758233
}
{
    "time": 33594.0,
    "id": 24873050
}
{
    "time": 38510.0,
    "id": 24861549
}
{"time":52708.0,"id":-9713871}{
    "time": 64200.0,
    "id": -9713870
}
{
    "time": 55388.0,
    "id": 25139379
}
{
    "time": 56755.0,
    "id

Dict{Any,Any} with 3 entries:
  "shifts"               => Any[Dict{Any,Any}("jobs"=>Any[Dict{Any,Any}("time"=…
  "route_cost"           => 26525.0
  "nb_assigned_bookings" => 25

In [49]:
JSON.print(output, 4)
{
    "julia": "0.5"
}

{
    "shifts": [
        {
            "jobs": [
                {
                    "time": 45000.0,
                    "id": -9714126
                },
                {
                    "time": 58800.0,
                    "id": -9714125
                },
                {
                    "time": 46800.0,
                    "id": 25139347
                },
                {
                    "time": 49942.0,
                    "id": 25060099
                },
                {
                    "time": 49702.0,
                    "id": 24244107
                },
                {
                    "time": 46325.0,
                    "id": 24601133
                },
                {
                    "time": 46800.0,
                    "id": 25139348
                },
                {
                    "time": 49942.0,
                    "id": 25060100
                },
                {
                    "time": 49702.0,
                    "id

ErrorException: syntax: { } vector syntax is discontinued